In [1]:
from cellcalc import get_primitive_hkl, MID, get_pri_vec_inplane
from interface_generator import core, print_near_axis, convert_vector_index,d_hkl,three_dot,cross_plane, get_right_hand
from csl_generator import print_list, getsigmas, get_theta_m_n_list
from numpy import array
from numpy import array, dot, round
from numpy.linalg import inv, norm
import numpy as np
import glob
import shutil
import os
from cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, MID
from interface_generator import core, print_near_axis, convert_vector_index, write_trans_file
from numpy import array, dot, round, cross, ceil
from numpy.linalg import inv, det, norm
from numpy import cross, dot, ceil
from numpy.linalg import norm, inv

import numpy as np
import pandas as pd
import shutil
import os
import math
import glob
from scipy.spatial import distance
import matplotlib.pyplot as plt
from numpy.linalg import inv, norm
from numpy import dot

from Element import Element

In [96]:
fcc_names = ["Ag","Al","Au","Cu","Ni","Pb","Pd","Pt"]
bcc_names = ["Fe","Mo","Nb","Ta","V","W"]

In [97]:
axises = ["100","110","111"]

In [14]:
def get_a_b(CSL, axis):
    hkl_perp_axis = MID(CSL, axis)
    a, b = get_pri_vec_inplane(hkl_perp_axis, CSL).T

    if(norm(cross(axis,[1,0,0])) < 1e-8):
        b = a + b
    elif (norm(cross(axis,[1,1,1])) < 1e-8):
        if dot(a,b) < 0:
            b = a + b
        b = a + b
    if (abs(norm(a) - norm(b)) < 1e-8):
        raise RuntimeError ('the tow vectors are identical!')

    return a.T, b.T

In [90]:
def get_expansion_xyz(cell):
    exp_x = ceil(100/norm(cell[:,0]))
    exp_y = ceil(20/norm(cell[:,1]))
    exp_z = ceil(20/norm(cell[:,2]))
    return exp_x, exp_y, exp_z

In [91]:
def define_bicrystal_regions(xhi):
    tol = 1e-5
    """
    generate a file defining some regions in the LAMMPS and define the atoms
    inside these regions into some groups.
    argument:
    region_names --- list of name of regions
    region_los --- list of the low bounds
    region_his --- list of the hi bounds
    """
    end_fixbulk1 = xhi/2-30
    start_fixbulk2 = xhi/2+30
    start_middle = xhi/2-20
    end_middle = xhi/2+20
    start_right = xhi/2 + tol
    start_bulk = 160
    end_bulk = 165


    with open('blockfile', 'w') as fb:
        fb.write('region fixbulk1 block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(end_fixbulk1))
        fb.write('region fixbulk2 block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
        format(start_fixbulk2))
        fb.write('region middle block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_middle,end_middle))
        fb.write('region right block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
        format(start_right))
        fb.write('region left block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_right))
        fb.write('region bulk block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_bulk,end_bulk))
        fb.write('group fixbulk1 region fixbulk1 \n')
        fb.write('group fixbulk2 region fixbulk2 \n')
        fb.write('group middle region middle \n')
        fb.write('group right region right \n')
        fb.write("group left region left \n")
        fb.write('group bulk region bulk \n')

In [92]:
def define_bicrystal_regions2(xhi):
    half_cell = xhi/2
    text = f"""

displace_atoms right move ${{tx}} ${{ty}} ${{tz}} units box

group left_dash region left 

group deleted_atoms subtract left_dash left

displace_atoms deleted_atoms move {half_cell} 0 0 units box

group fixbulk1_dash region fixbulk1 
group fixbulk2_dash region fixbulk2 
group middle_dash region middle 
group bulk_dash region bulk
            """
    with open("blockfile2", "w") as f:
        f.writelines(text)

In [93]:
def get_thetas(axis,max_sigma):
    theta_list = []
    sigma_list = []
    lists, thetas = getsigmas(axis, max_sigma)
    return thetas

In [15]:
class core2(core):
    def __init__(self, file_1, file_2):
        super().__init__(file_1,file_2)
    def compute_stgb(self, axis, plane_n, lim = 20, tol = 1e-10):
        """
        compute the transformation to obtain the supercell of the two slabs forming a interface
        argument:
        hkl --- miller indices of the plane expressed in lattice_1
        lim --- the limit searching for a CSL vector cross the plane
        normal_ortho --- whether limit the vector crossing the GB to be normal to the GB
        plane_ortho --- whether limit the two vectors in the GB plane to be orthogonal
        tol --- tolerance judging whether orthogonal
        """
        hkl = MID(self.lattice_1,plane_n)
        self.d1 = d_hkl(self.lattice_1, hkl)
        lattice_2 = three_dot(self.R, self.D, self.lattice_2)
        hkl_2 = get_primitive_hkl(hkl, self.lattice_1, lattice_2)
        self.d2 = d_hkl(lattice_2, hkl_2)
        hkl_c = get_primitive_hkl(hkl, self.lattice_1, self.CSL) # miller indices of the plane in CSL
        hkl_c = np.array(hkl_c)
        
        #plane_n = dot(self.lattice_1,hkl)
        axis_cartesian = dot(self.conv_lattice_1,axis)
        v1 = cross_plane(lattice=self.CSL, n=plane_n, lim=lim,  orthogonal=True, tol=tol) # a CSL basic vector cross the plane
        v2 = cross_plane(lattice=self.CSL, n=axis_cartesian, lim=lim, orthogonal=True, tol=tol)
        v3_cartesian = cross(v1,v2)
        v3 = cross_plane(lattice=self.CSL, n=v3_cartesian, lim=lim, orthogonal=True, tol=tol)
        supercell = np.column_stack((v1,v2,v3)) # supercell of the bicrystal
        supercell = get_right_hand(supercell) # check right-handed


        self.bicrystal_U1 = np.array(np.round(dot(inv(self.lattice_1), supercell),8),dtype = int)
        self.bicrystal_U2 = np.array(np.round(dot(inv(self.lattice_2_TD), supercell),8),dtype = int)
        self.cell_calc.compute_CNID(hkl)
        CNID = self.cell_calc.CNID
        self.CNID = dot(self.lattice_1, CNID)

In [94]:
def get_gb_files(interface,axis,hkl,element):
    interface.compute_stgb(axis, hkl, lim = 10, tol = 1e-5)
    half_cell = dot(interface.lattice_1, interface.bicrystal_U1)
    x,y,z = get_expansion_xyz(half_cell)
    xhi = 2*x*norm(half_cell[:,0])
    
#     hkl_name = get_primitive_hkl(hkl, interface.lattice_1, interface.conv_lattice_1)
    hkl_name = np.array(np.round(dot(inv(interface.conv_lattice_1), hkl),8),dtype = int)
    hkl_name = np.array(hkl_name, dtype=int)
    hkl_name_num = hkl_name.copy()
    hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
    hkl_x, hkl_y, hkl_z = hkl_name
    axis_x, axis_y, axis_z = axis
    dirname = f"{axis_x}{axis_y}{axis_z}_{str(hkl_x)}_{str(hkl_y)}_{str(hkl_z)}_gb"
    os.makedirs(dirname,exist_ok=True)
    os.chdir(dirname) 
    
    if (axis == [1,1,1]):    
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = True)
    else:
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False)

    define_bicrystal_regions(xhi)
    define_bicrystal_regions2(xhi)
    
    with open(f"{element.element}.meam","w") as f:
        f.writelines(element.meam())

    with open("library.meam","w") as f:
        f.writelines(element.library_meam())

    with open("proto.in","w") as f:
        f.writelines(element.proto())


    os.chdir(os.pardir)

In [95]:
def get_all_STGBs(axis,element,max_sigma):

    cif_filename = glob.glob(f"/homenfs1/yhata/python_script/{element.element}/*.cif")[0]

    my_interface = core2(cif_filename, cif_filename)
    my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
    factor = element.getLatP()/(norm(my_interface.conv_lattice_1[:,0]))
    my_interface.lattice_1 =  my_interface.lattice_1*factor
    my_interface.lattice_2 =  my_interface.lattice_2*factor
    my_interface.conv_lattice_1 =  my_interface.conv_lattice_1*factor
    my_interface.conv_lattice_2 =  my_interface.conv_lattice_2*factor  
    my_interface.lattice_1 = my_interface.conv_lattice_1
    my_interface.lattice_2 = my_interface.conv_lattice_2
    
    my_interface2 = core2(cif_filename, cif_filename)
    my_interface2.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
    factor = element.getLatP()/(norm(my_interface2.conv_lattice_1[:,0]))
    my_interface2.lattice_1 =  my_interface2.lattice_1*factor
    my_interface2.lattice_2 =  my_interface2.lattice_2*factor
    my_interface2.conv_lattice_1 =  my_interface2.conv_lattice_1*factor
    my_interface2.conv_lattice_2 =  my_interface2.conv_lattice_2*factor
    axis_cart = dot(my_interface2.conv_lattice_1, np.array(axis))

    axis_pri = np.array(np.round(dot(inv(my_interface2.lattice_1), axis_cart),8),dtype = int)

    thetas = get_thetas(axis,max_sigma)


    for theta in thetas:
        
        my_interface.search_one_position(np.array(axis),theta-0.0000001,1,0.0000001)
        my_interface2.search_one_position(axis_pri,theta-0.0000001,1,0.0000001)
        n_1, n_2 = get_a_b(my_interface.CSL, dot(my_interface.lattice_1,np.array(axis)))
        
        get_gb_files(my_interface2,axis,n_1,element)
        get_gb_files(my_interface2,axis,n_2,element)

In [ ]:
for name in bcc_names:
    element = Element(name)
    os.makedirs(name)
    os.chdir(name)
    for axis_name in axises:
        os.makedirs(axis_name)
        os.chdir(axis_name)
        axis = list(map(int, list(axis_name)))
        get_all_STGBs(axis,element,max_sigma)
        os.chdir("..")
    os.chdir("..")

In [47]:
axis = np.array([1,0,0])
max_sigma = 100
eps = 1e-5

element = Element("Cu")
cif_filename = "Cu_mp-30_conventional_standard.cif"
my_interface = core2(cif_filename, cif_filename)
my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
factor = element.getLatP()/(norm(my_interface.conv_lattice_1[:,0]))
my_interface.lattice_1 =  my_interface.lattice_1*factor
my_interface.lattice_2 =  my_interface.lattice_2*factor
my_interface.conv_lattice_1 =  my_interface.conv_lattice_1*factor
my_interface.conv_lattice_2 =  my_interface.conv_lattice_2*factor  
my_interface.lattice_1 = my_interface.conv_lattice_1
my_interface.lattice_2 = my_interface.conv_lattice_2


Warning!, this programme will rewrite the POSCAR file in this dir!


In [48]:
cif_filename = "Cu_mp-30_conventional_standard.cif"
my_interface2 = core2(cif_filename, cif_filename)
my_interface2.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
factor = element.getLatP()/(norm(my_interface2.conv_lattice_1[:,0]))
my_interface2.lattice_1 =  my_interface2.lattice_1*factor
my_interface2.lattice_2 =  my_interface2.lattice_2*factor
my_interface2.conv_lattice_1 =  my_interface2.conv_lattice_1*factor
my_interface2.conv_lattice_2 =  my_interface2.conv_lattice_2*factor
axis_cart = dot(my_interface2.conv_lattice_1, axis)

axis_pri = np.array(np.round(dot(inv(my_interface2.lattice_1), axis_cart),8),dtype = int)

Warning!, this programme will rewrite the POSCAR file in this dir!


In [43]:
getsigmas(axis.tolist(), max_sigma)

([5, 13, 17, 25, 29, 37, 41, 53, 61, 65, 73, 85, 89, 97],
 [36.86989764584402,
  22.61986494804043,
  28.072486935852957,
  16.260204708311957,
  43.60281897270362,
  18.924644416051233,
  12.680383491819818,
  31.890791801845708,
  10.388857815469612,
  14.250032697803595,
  41.112090439166934,
  8.797410709991064,
  25.989233583833013,
  42.075022050843636])

In [49]:
theta = 42.075022050843636
my_interface.search_one_position(axis,theta-0.0000001,1,0.0000001)
my_interface2.search_one_position(axis_pri,theta-0.0000001,1,0.0000001)

CSL = my_interface.CSL
n_1, n_2 = get_a_b(my_interface.CSL, dot(my_interface.lattice_1,axis))
hkl_1, hkl_2 = get_STGB_MLs(my_interface2.lattice_1, n_1, n_2)
print(n_1,n_2)

[3.6200000e+00 0.0000000e+00 2.2166107e-16]
rotation matrix = [[ 1.00000000e+00 -4.10319803e-17  1.57815308e-17]
 [ 4.10319803e-17  7.42268042e-01 -6.70103091e-01]
 [ 1.57815308e-17  6.70103091e-01  7.42268042e-01]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  0]
 [ 0  4 -9]
 [ 0  9  4]]; sigma_1 = 97

U2 = 
[[ 1  0  0]
 [ 0  9 -4]
 [ 0  4  9]]; sigma_2 = 97

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]

axis = [3.6200000e+00 0.0000000e+00 2.2166107e-16] ; theta = 42.07502195

[3.62 0.   0.  ]
rotation matrix = [[ 1.          0.          0.        ]
 [ 0.          0.74226804 -0.67010309]
 [ 0.          0.67010309  0.74226804]]
Congrates, we found an appx CSL!

U1 = 
[[-1 -2 -6]
 [-1  3  7]
 [ 1  6 -3]]; sigma_1 = 97

U2 = 
[[-1  3 -6]
 [-1 -2  7]
 [ 1  6  2]]; sigma_2 = 97

D = 
[[ 1. -0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 0.   0.  ] ; theta = 42.07502195

[-8.86644281e-16  1.44800000e+01  3.25800000e+01] [ 1.10830535e-15 -1.81000000e+01  4.70599999e+01]


In [51]:
my_interface2.compute_stgb(axis, n_2, lim = 10, tol = 1e-5)
my_interface2.get_bicrystal(xyz_1 = [1,1,1], xyz_2 =[1,1,1],filename = 'atominfile', filetype='LAMMPS',mirror = False)

[ 0.         -9.04999999 23.52999997]
[-3.62  0.    0.  ]
[-1.33226763e-15  2.35300000e+01  9.04999999e+00]
[[ 9  1  4]
 [-9  1 -4]
 [-4 -1  9]]
[[ 4  1  9]
 [-4  1 -9]
 [-9 -1  4]]
num of atoms1 = 194
[[ 2.52103827e+01  2.23076085e-16 -2.36129496e-16]
 [-1.66622796e-31  3.62000000e+00 -9.97474817e-16]
 [ 7.60858304e-16  0.00000000e+00  2.52103827e+01]]


In [78]:
def get_gb_files(interface,axis,hkl,element):
    interface.compute_stgb(axis, hkl, lim = 10, tol = 1e-5)
    half_cell = dot(interface.lattice_1, interface.bicrystal_U1)
    x,y,z = get_expansion_xyz(half_cell)
    xhi = 2*x*norm(half_cell[:,0])
    
#     hkl_name = get_primitive_hkl(hkl, interface.lattice_1, interface.conv_lattice_1)
    hkl_name = np.array(np.round(dot(inv(interface.conv_lattice_1), hkl),8),dtype = int)
    hkl_name = np.array(hkl_name, dtype=int)
    hkl_name_num = hkl_name.copy()
    hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
    hkl_x, hkl_y, hkl_z = hkl_name
    axis_x, axis_y, axis_z = axis
    dirname = f"{axis_x}{axis_y}{axis_z}_{str(hkl_x)}_{str(hkl_y)}_{str(hkl_z)}_gb"
    os.makedirs(dirname,exist_ok=True)
    os.chdir(dirname) 
    
    if (axis == [1,1,1]):    
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = True)
    else:
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False)

    define_bicrystal_regions(xhi)
    define_bicrystal_regions2(xhi)
    
    with open(f"{element.element}.meam","w") as f:
        f.writelines(element.meam())

    with open("library.meam","w") as f:
        f.writelines(element.library_meam())

    with open("proto.in","w") as f:
        f.writelines(element.proto())


    os.chdir(os.pardir)

In [79]:
def get_all_STGBs(axis,element,max_sigma):

    cif_filename = glob.glob(f"/homenfs1/yhata/python_script/{element.element}/*.cif")[0]

    my_interface = core2(cif_filename, cif_filename)
    my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
    factor = element.getLatP()/(norm(my_interface.conv_lattice_1[:,0]))
    my_interface.lattice_1 =  my_interface.lattice_1*factor
    my_interface.lattice_2 =  my_interface.lattice_2*factor
    my_interface.conv_lattice_1 =  my_interface.conv_lattice_1*factor
    my_interface.conv_lattice_2 =  my_interface.conv_lattice_2*factor  
    my_interface.lattice_1 = my_interface.conv_lattice_1
    my_interface.lattice_2 = my_interface.conv_lattice_2
    
    my_interface2 = core2(cif_filename, cif_filename)
    my_interface2.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
    factor = element.getLatP()/(norm(my_interface2.conv_lattice_1[:,0]))
    my_interface2.lattice_1 =  my_interface2.lattice_1*factor
    my_interface2.lattice_2 =  my_interface2.lattice_2*factor
    my_interface2.conv_lattice_1 =  my_interface2.conv_lattice_1*factor
    my_interface2.conv_lattice_2 =  my_interface2.conv_lattice_2*factor
    axis_cart = dot(my_interface2.conv_lattice_1, np.array(axis))

    axis_pri = np.array(np.round(dot(inv(my_interface2.lattice_1), axis_cart),8),dtype = int)

    thetas = get_thetas(axis,max_sigma)


    for theta in thetas:
        
        my_interface.search_one_position(np.array(axis),theta-0.0000001,1,0.0000001)
        my_interface2.search_one_position(axis_pri,theta-0.0000001,1,0.0000001)
        n_1, n_2 = get_a_b(my_interface.CSL, dot(my_interface.lattice_1,np.array(axis)))
        
        get_gb_files(my_interface2,axis,n_1,element)
        get_gb_files(my_interface2,axis,n_2,element)

In [89]:
os.getcwd()

'/homenfs1/yhata/calc/220429/make_bicrystal'

In [88]:
os.chdir("..")

In [80]:
bcc_names = ["Fe","Mo","Nb","Ta","V","W"]

In [81]:
axises = ["100","110","111"]

In [82]:
for name in bcc_names:
    element = Element(name)
    os.makedirs(name)
    os.chdir(name)
    for axis_name in axises:
        os.makedirs(axis_name)
        os.chdir(axis_name)
        axis = list(map(int, list(axis_name)))
        get_all_STGBs(axis,element,max_sigma)
        os.chdir("..")
    os.chdir("..")

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[2.85100016e+00 0.00000000e+00 1.74573411e-16]
rotation matrix = [[ 1.00000000e+00 -3.67394039e-17  1.22464679e-17]
 [ 3.67394039e-17  8.00000001e-01 -5.99999999e-01]
 [ 1.22464679e-17  5.99999999e-01  8.00000001e-01]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  0]
 [ 0  1 -2]
 [ 0  2  1]]; sigma_1 = 5

U2 = 
[[ 1  0  0]
 [ 0  2 -1]
 [ 0  1  2]]; sigma_2 = 5

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]

axis = [2.85100016e+00 0.00000000e+00 1.74573411e-16] ; theta = 36.86989755

[2.85100016e+00 0.00000000e+00 2.22044605e-16]
rotation matrix = [[ 1.00000000e+00 -4.67298334e-17  1.55766111e-17]
 [ 4.67298334e-17  8.00000001e-01 -5.99999999e-01]
 [ 1.55766111e-17  5.99999999e-01  8.00000001e-01]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  1  1]
 [ 1  2 -1]
 [ 0  2  1]]; sigma_1 = 5

U2 = 
[[1 0 1]
 [1 2 0]
 [0 1 2]]; sigma_2 = 5

D = 
[[ 1.  

[[ 1.07471740e+02  1.64930592e-15  2.26635146e-15]
 [-6.36443059e-32  2.28080013e+01 -3.55271368e-15]
 [ 5.88492399e-15  1.97215226e-31  3.07062115e+01]]
[ 4.44089210e-16 -8.55300048e+00 -1.99570011e+01]
[-2.85100016e+00  0.00000000e+00 -2.22044605e-16]
[-2.22044605e-15  1.99570011e+01 -8.55300048e+00]
[[-3 -1  7]
 [ 7 -1  3]
 [ 4  0 10]]
[[-7 -1  3]
 [ 3 -1  7]
 [-4  0 10]]
num of atoms1 = 116
[[ 1.08562852e+02  6.99741240e-16 -5.95740499e-16]
 [ 8.82356517e-31  2.28080013e+01  2.88657986e-15]
 [ 5.35414635e-15  9.86076132e-32  2.17125703e+01]]
[2.85100016e+00 0.00000000e+00 1.74573411e-16]
rotation matrix = [[ 1.00000000e+00 -1.98591372e-17  3.30985618e-18]
 [ 1.98591372e-17  9.45945947e-01 -3.24324323e-01]
 [ 3.30985618e-18  3.24324323e-01  9.45945947e-01]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  0]
 [ 0 -6  1]
 [ 0 -1 -6]]; sigma_1 = 37

U2 = 
[[ 1  0  0]
 [ 0 -6 -1]
 [ 0  1 -6]]; sigma_2 = 37

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [2.85100016e+00 0.00

[[ 1.14927491e+02  2.20329949e-15 -1.32172251e-15]
 [ 4.68281230e-31  2.28080013e+01  2.88657986e-15]
 [-1.91898160e-15 -3.94430453e-31  2.29854981e+01]]
[-1.33226763e-15  1.99570011e+01  2.56590014e+01]
[2.85100016e+00 0.00000000e+00 2.22044605e-16]
[ 1.77635684e-15 -2.56590014e+01  1.99570011e+01]
[[ 7  1  9]
 [-9  1  7]
 [-2  0 16]]
[[ 9  1  7]
 [-7  1  9]
 [ 2  0 16]]
num of atoms1 = 260
[[ 1.30025613e+02 -1.55796801e-16 -3.50389630e-15]
 [-1.17962573e-30  2.28080013e+01 -1.55431223e-15]
 [-4.54338926e-15 -1.97215226e-31  3.25064032e+01]]
[2.85100016e+00 0.00000000e+00 1.74573411e-16]
rotation matrix = [[ 1.00000000e+00 -4.02623604e-17  1.50983851e-17]
 [ 4.02623604e-17  7.53424659e-01 -6.57534245e-01]
 [ 1.50983851e-17  6.57534245e-01  7.53424659e-01]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  0]
 [ 0 -8  3]
 [ 0 -3 -8]]; sigma_1 = 73

U2 = 
[[ 1  0  0]
 [ 0 -8 -3]
 [ 0  3 -8]]; sigma_2 = 73

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]

axis = [2.85100016e+00 0.00000

rotation matrix = [[ 0.66666667  0.33333333  0.66666667]
 [ 0.33333333  0.66666667 -0.66666667]
 [-0.66666667  0.66666667  0.33333333]]
Congrates, we found an appx CSL!

U1 = 
[[ 0 -1 -2]
 [-1  0 -1]
 [ 0  1 -1]]; sigma_1 = 3

U2 = 
[[ 0 -1 -2]
 [ 0  1 -1]
 [ 1  0 -1]]; sigma_2 = 3

D = 
[[ 1.  0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [2.85100016e+00 2.85100016e+00 4.44089210e-16] ; theta = 70.52877927

[ 1.42550008 -1.42550008 -1.42550008]
[2.85100016e+00 2.85100016e+00 4.44089210e-16]
[ 2.85100016 -2.85100016  5.70200032]
[[ 0  2  0]
 [ 1  1 -1]
 [ 0  1 -3]]
[[ 0  2  0]
 [ 0  1 -3]
 [-1  1 -1]]
num of atoms1 = 6
[[ 1.01230581e+02 -1.18750156e-15  7.13989667e-16]
 [-2.25408353e-15  2.01596155e+01  5.18506506e-16]
 [ 1.02359822e-15  1.31535393e-15  2.09504870e+01]]
[ 2.85100016 -2.85100016  5.70200032]
[2.85100016e+00 2.85100016e+00 4.44089210e-16]
[ 1.42550008 -1.42550008 -1.42550008]
[[ 0  2  0]
 [-1  1 -1]
 [-3  1  0]]
[[ 0  2  0]
 [-3  1  0]
 [-1  1  1]]
num of atoms1 = 6
[[ 

[[ 1.03699620e+02 -1.90434589e-15 -2.05691366e-15]
 [ 6.19825031e-16  2.01596155e+01 -2.94079916e-16]
 [-1.73810676e-15 -2.08495482e-15  2.09504870e+01]]
[-14.2550008   14.2550008    5.70200032]
[2.85100016e+00 2.85100016e+00 4.44089210e-16]
[-1.42550008  1.42550008 -7.1275004 ]
[[ 0  2  0]
 [-7  1  2]
 [ 3  1  3]]
[[ 0  2  0]
 [-3  1  3]
 [ 7  1  2]]
num of atoms1 = 54
[[ 1.04752435e+02  1.14864155e-16  3.55579176e-15]
 [ 2.61978378e-15  2.01596155e+01 -2.43871937e-15]
 [-4.86815824e-16 -1.76422510e-16  2.22213471e+01]]
[2.85100016e+00 2.85100016e+00 3.49146822e-16]
rotation matrix = [[ 0.96969697  0.03030303  0.24242424]
 [ 0.03030303  0.96969697 -0.24242424]
 [-0.24242424  0.24242424  0.93939394]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0 -4]
 [ 1  1  4]
 [ 0 -4  1]]; sigma_1 = 33

U2 = 
[[ 1  1 -4]
 [ 1  0  4]
 [ 0 -4 -1]]; sigma_2 = 33

D = 
[[ 1.  0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [2.85100016e+00 2.85100016e+00 3.49146822e-16] ; theta = 20.04997562

[2.85100016

...

[[ 1.03734726e+02  2.02231645e-15  1.79194613e-15]
 [ 1.48588157e-14  2.23720063e+01 -3.35150205e-15]
 [ 3.50963135e-15  3.81227731e-15  2.93406112e+01]]
[ 15.81939737 -15.81939737 -18.98327685]
[-3.16387947e+00 -3.16387947e+00 -4.44089210e-16]
[ 4.74581921 -4.74581921  7.90969869]
[[ 0 -2  0]
 [11 -1  1]
 [ 1 -1  4]]
[[  0  -2   0]
 [ -1  -1   4]
 [-11  -1   1]]
num of atoms1 = 86
[[ 1.17362445e+02  2.79614547e-16  1.07443057e-15]
 [-1.56401194e-15  2.23720063e+01 -3.28782726e-16]
 [ 8.36791127e-15 -2.66010529e-16  2.07469452e+01]]
[3.16387947e+00 3.16387947e+00 3.87463487e-16]
rotation matrix = [[ 0.98039216  0.01960784  0.19607843]
 [ 0.01960784  0.98039216 -0.19607843]
 [-0.19607843  0.19607843  0.96078431]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -1 -5]
 [ 1  0  5]
 [ 0 -5  1]]; sigma_1 = 51

U2 = 
[[ 1  0 -5]
 [ 1 -1  5]
 [ 0 -5 -1]]; sigma_2 = 51

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.16387947e+00 3.16387947e+00 3.87463487e-16] ; theta = 16.09893385


[[ 1.08128792e+02  3.86753194e-15 -5.71486080e-16]
 [ 1.08457135e-14  2.23720063e+01 -2.80178740e-15]
 [ 5.44418305e-16 -1.76271635e-17  3.82293012e+01]]
[ -3.16387947   3.16387947 -37.9665537 ]
[3.16387947e+00 3.16387947e+00 4.44089210e-16]
[-18.98327685  18.98327685   3.16387947]
[[ 0  2  0]
 [11  1  7]
 [13  1 -5]]
[[ 0  2  0]
 [13  1  5]
 [11  1 -7]]
num of atoms1 = 292
[[ 1.14687903e+02 -1.56191394e-15  3.07984634e-16]
 [ 2.37622626e-16  2.23720063e+01 -4.27237392e-15]
 [-1.64257393e-15 -3.34869181e-16  2.70321981e+01]]
[3.16387947e+00 3.16387947e+00 3.87463487e-16]
rotation matrix = [[ 0.60493827  0.39506173  0.69135802]
 [ 0.39506173  0.60493827 -0.69135802]
 [-0.69135802  0.69135802  0.20987654]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -4  4]
 [ 1  3 -4]
 [ 0  4  7]]; sigma_1 = 81

U2 = 
[[ 1 -4 -4]
 [ 1  3  4]
 [ 0 -4  7]]; sigma_2 = 81

D = 
[[ 1.  0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.16387947e+00 3.16387947e+00 3.87463487e-16] ; theta = 77.88488244

[3.1638

[[ 1.10180712e+02  2.72017647e-15  1.97157683e-15]
 [-1.07798556e-15  2.23720063e+01 -3.97528133e-15]
 [-9.38642237e-16 -1.52411038e-16  4.45197305e+01]]
[ 3.16387947 -3.16387947 44.29431265]
[3.16387947e+00 3.16387947e+00 4.44089210e-16]
[-11.07357816  11.07357816   1.58193974]
[[  0   2   0]
 [-13   1  -4]
 [-15   1   3]]
[[  0   2   0]
 [-15   1  -3]
 [-13   1   4]]
num of atoms1 = 198
[[ 1.33559191e+02  1.92375515e-15 -1.31688572e-15]
 [ 1.54340741e-15  2.23720063e+01  2.50401977e-15]
 [-2.15872369e-15  9.23768715e-16  3.14802033e+01]]
Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.16387947 3.16387947 3.16387947]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  1  1]
 [ 0 -1  1]
 [ 1  0  1]]; sigma_1 = 3

U2 = 
[[-1  0  1]
 [ 1 -1  1]
 [ 0  1  1]]; sigma_2 = 3

D

[[ 1.24562059e+02  8.27826686e-16 -6.72807430e-15]
 [ 1.74712879e-15  2.19200000e+01 -6.24328965e-15]
 [-2.77411930e-15 -1.21927296e-15  4.31495631e+01]]
[ 22.14715632 -34.80267422  12.6555179 ]
[-1.58193974 -1.58193974 -1.58193974]
[-15.81939737  -3.16387947  18.98327685]
[[ -4  -1   6]
 [  3   0  11]
 [-15   0   7]]
[[ -7  -1   5]
 [ -3   0  11]
 [-18   0   4]]
num of atoms1 = 186
[[ 1.29448689e+02  2.14636074e-15  2.64728645e-16]
 [-5.43550964e-15  2.19200000e+01  9.41397982e-16]
 [-6.75194068e-15  3.46330830e-15  2.49124119e+01]]
[3.16387947 3.16387947 3.16387947]
rotation matrix = [[ 0.75675676 -0.32432432  0.56756757]
 [ 0.56756757  0.75675676 -0.32432432]
 [-0.32432432  0.56756757  0.75675676]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  4 -3]
 [ 1  0  4]
 [ 1 -3  0]]; sigma_1 = 37

U2 = 
[[ 1  4  0]
 [ 1 -3  4]
 [ 1  0 -3]]; sigma_2 = 37

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.16387947 3.16387947 3.16387947] ; theta = 50.56999199

[3.16387947 3.16387947

[[ 1.26871172e+02 -2.83020582e-15 -1.07384941e-15]
 [-2.98475270e-15  2.19200000e+01 -6.74116749e-15]
 [ 3.88241653e-15  8.03370198e-16  3.66245528e+01]]
[3.16387947 3.16387947 3.16387947]
rotation matrix = [[ 0.98630137 -0.10958904  0.12328767]
 [ 0.12328767  0.98630137 -0.10958904]
 [-0.10958904  0.12328767  0.98630137]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -6 -2]
 [ 1  2 -3]
 [ 1  3  6]]; sigma_1 = 73

U2 = 
[[ 1 -6 -3]
 [ 1  3 -2]
 [ 1  2  6]]; sigma_2 = 73

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.16387947 3.16387947 3.16387947] ; theta = 11.63505119

[3.16387947 3.16387947 3.16387947]
rotation matrix = [[ 0.98630137 -0.10958904  0.12328767]
 [ 0.12328767  0.98630137 -0.10958904]
 [-0.10958904  0.12328767  0.98630137]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -3 -2]
 [ 0 -9  1]
 [ 0 -1 -8]]; sigma_1 = 73

U2 = 
[[ 1 -2 -3]
 [ 0 -8 -1]
 [ 0  1 -9]]; sigma_2 = 73

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.16387947 3.16387947 3.163

In [85]:
for name in fcc_names:
    element = Element(name)
    os.makedirs(name)
    os.chdir(name)
    for axis_name in axises:
        os.makedirs(axis_name)
        os.chdir(axis_name)
        axis = list(map(int, list(axis_name)))
        get_all_STGBs(axis,element,max_sigma)
        os.chdir("..")
    os.chdir("..")

Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[4.07293504e+00 0.00000000e+00 2.49395343e-16]
rotation matrix = [[ 1.00000000e+00 -3.67394039e-17  1.22464679e-17]
 [ 3.67394039e-17  8.00000001e-01 -5.99999999e-01]
 [ 1.22464679e-17  5.99999999e-01  8.00000001e-01]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  0]
 [ 0  1 -2]
 [ 0  2  1]]; sigma_1 = 5

U2 = 
[[ 1  0  0]
 [ 0  2 -1]
 [ 0  1  2]]; sigma_2 = 5

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]

axis = [4.07293504e+00 0.00000000e+00 2.49395343e-16] ; theta = 36.86989755

[4.07293504 0.         0.        ]
rotation matrix = [[ 1.   0.   0. ]
 [ 0.   0.8 -0.6]
 [ 0.   0.6  0.8]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  1]
 [ 1  1 -2]
 [-1  1  1]]; sigma_1 = 5

U2 = 
[[ 1  1  1]
 [ 1  0 -2]
 [-1  1  0]]; sigma_2 = 5

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]

axis = [4.07293504 0.         0.        ] ; theta = 36.8698975

[[ 1.08564922e+02 -8.18680322e-16  2.65801580e-15]
 [ 8.33554418e-31  2.03646752e+01  4.98790686e-16]
 [ 5.36222171e-15 -5.76795681e-32  3.10185492e+01]]
[4.07293504e+00 0.00000000e+00 2.49395343e-16]
rotation matrix = [[ 1.00000000e+00 -1.98591372e-17  3.30985618e-18]
 [ 1.98591372e-17  9.45945947e-01 -3.24324323e-01]
 [ 3.30985618e-18  3.24324323e-01  9.45945947e-01]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  0]
 [ 0 -6  1]
 [ 0 -1 -6]]; sigma_1 = 37

U2 = 
[[ 1  0  0]
 [ 0 -6 -1]
 [ 0  1 -6]]; sigma_2 = 37

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]

axis = [4.07293504e+00 0.00000000e+00 2.49395343e-16] ; theta = 18.92464432

[4.07293504 0.         0.        ]
rotation matrix = [[ 1.          0.          0.        ]
 [ 0.          0.94594595 -0.32432432]
 [ 0.          0.32432432  0.94594595]]
Congrates, we found an appx CSL!

U1 = 
[[-1 -3 -2]
 [-1  4  3]
 [ 1  2 -4]]; sigma_1 = 37

U2 = 
[[-1 -2 -3]
 [-1  3  4]
 [ 1  3 -3]]; sigma_2 = 37

D = 
[[ 1. -0. -0.]
 [-0.  

[[ 1.16096511e+02 -8.74936743e-16  2.13286294e-15]
 [ 5.11251078e-31  2.03646752e+01  1.24697672e-16]
 [ 4.56666021e-15 -4.95346148e-32  2.32193022e+01]]
[4.07293504e+00 0.00000000e+00 2.49395343e-16]
rotation matrix = [[ 1.00000000e+00 -4.02623604e-17  1.50983851e-17]
 [ 4.02623604e-17  7.53424659e-01 -6.57534245e-01]
 [ 1.50983851e-17  6.57534245e-01  7.53424659e-01]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  0]
 [ 0 -8  3]
 [ 0 -3 -8]]; sigma_1 = 73

U2 = 
[[ 1  0  0]
 [ 0 -8 -3]
 [ 0  3 -8]]; sigma_2 = 73

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [ 0.  0.  1.]]

axis = [4.07293504e+00 0.00000000e+00 2.49395343e-16] ; theta = 41.11209034

[4.07293504 0.         0.        ]
rotation matrix = [[ 1.          0.          0.        ]
 [ 0.          0.75342466 -0.65753425]
 [ 0.          0.65753425  0.75342466]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  5 -2]
 [ 1 -6  3]
 [-1 -2 -6]]; sigma_1 = 73

U2 = 
[[ 1  2 -5]
 [ 1 -3  6]
 [-1 -5 -3]]; sigma_2 = 73

D = 
[[ 1. -0.  0.]
 [ 0.  

[[ 1.05817956e+02  1.09437987e-15 -2.94573496e-16]
 [ 2.61257093e-15  2.01599999e+01 -2.62000864e-16]
 [ 1.13756453e-15  3.11392734e-17  2.49415315e+01]]
[ 2.03646752 -2.03646752  4.07293504]
[-2.03646752 -2.03646752  0.        ]
[-4.07293504  4.07293504  4.07293504]
[[ 1  1 -1]
 [-2  0 -1]
 [ 0  0  3]]
[[ 1  1  1]
 [ 0  0 -3]
 [-2  0  1]]
num of atoms1 = 6
[[ 1.04754432e+02  3.11392734e-17  2.06447406e-15]
 [-9.54175948e-17  2.01599999e+01  3.31616106e-16]
 [ 2.67423393e-15  4.88273325e-16  2.11635913e+01]]
[4.07293504e+00 4.07293504e+00 4.98790686e-16]
rotation matrix = [[ 0.88888889  0.11111111  0.44444444]
 [ 0.11111111  0.88888889 -0.44444444]
 [-0.44444444  0.44444444  0.77777778]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -1 -2]
 [ 1  0  2]
 [ 0 -2  1]]; sigma_1 = 9

U2 = 
[[ 1  0 -2]
 [ 1 -1  2]
 [ 0 -2 -1]]; sigma_2 = 9

D = 
[[ 1.  0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.07293504e+00 4.07293504e+00 4.98790686e-16] ; theta = 38.94244117

[4.07293504 4.07293504 0. 

[[ 1.04754432e+02  1.26697887e-15  1.98133712e-15]
 [-1.19842476e-15  2.01599999e+01  1.93618188e-16]
 [ 4.16447595e-15  1.41393224e-18  2.11635913e+01]]
[4.07293504e+00 4.07293504e+00 4.98790686e-16]
rotation matrix = [[ 0.96969697  0.03030303  0.24242424]
 [ 0.03030303  0.96969697 -0.24242424]
 [-0.24242424  0.24242424  0.93939394]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0 -4]
 [ 1  1  4]
 [ 0 -4  1]]; sigma_1 = 33

U2 = 
[[ 1  1 -4]
 [ 1  0  4]
 [ 0 -4 -1]]; sigma_2 = 33

D = 
[[ 1.  0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.07293504e+00 4.07293504e+00 4.98790686e-16] ; theta = 20.04997562

[4.07293504 4.07293504 0.        ]
rotation matrix = [[ 0.96969697  0.03030303  0.24242424]
 [ 0.03030303  0.96969697 -0.24242424]
 [-0.24242424  0.24242424  0.93939394]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  1  3]
 [ 0 -6  1]
 [ 0  3 -6]]; sigma_1 = 33

U2 = 
[[ 1  2  2]
 [ 0 -6  3]
 [ 0  1 -6]]; sigma_2 = 33

D = 
[[ 1. -0.  0.]
 [-0.  1. -0.]
 [-0.  0.  1.]]

axis = [4.0

...

[[ 1.02751693e+02 -4.74865658e-15 -3.49752988e-15]
 [ 5.71823210e-15  2.21599999e+01 -1.79047776e-15]
 [ 1.18608157e-14  1.28009870e-15  3.63282093e+01]]
[-9.79342889  9.79342889 11.75211467]
[1.95868578e+00 1.95868578e+00 4.37456136e-16]
[-11.75211467  11.75211467 -19.58685779]
[[ 3 -1 -5]
 [ 2  0 11]
 [-8  0 -1]]
[[-3 -1 -5]
 [ 8  0 -1]
 [-2  0 11]]
num of atoms1 = 86
[[ 1.08984628e+02  2.23592058e-15 -7.90625884e-16]
 [-8.62638598e-15  2.21599999e+01 -1.08832849e-14]
 [-2.99521146e-15  5.62066736e-16  2.56879232e+01]]
[3.91737156e+00 3.91737156e+00 4.79739654e-16]
rotation matrix = [[ 0.98039216  0.01960784  0.19607843]
 [ 0.01960784  0.98039216 -0.19607843]
 [-0.19607843  0.19607843  0.96078431]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -1 -5]
 [ 1  0  5]
 [ 0 -5  1]]; sigma_1 = 51

U2 = 
[[ 1  0 -5]
 [ 1 -1  5]
 [ 0 -5 -1]]; sigma_2 = 51

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156e+00 3.91737156e+00 4.79739654e-16] ; theta = 16.09893385

[3.91737156

[[ 1.00410112e+02 -4.03800760e-15  1.95654470e-15]
 [-9.80133511e-15  2.21599999e+01  4.46539826e-15]
 [-5.63500622e-16  1.08054863e-17  2.36668903e+01]]
[ 1.95868578 -1.95868578 23.50422935]
[1.95868578e+00 1.95868578e+00 4.37456136e-16]
[ 23.50422935 -23.50422935  -3.91737156]
[[  6  -1   1]
 [ -7   0  11]
 [ -5   0 -13]]
[[  6  -1  -1]
 [ -5   0  13]
 [ -7   0 -11]]
num of atoms1 = 146
[[ 1.18334452e+02  3.75804297e-15 -2.16432984e-15]
 [ 5.50097076e-16  2.21599999e+01  3.72545312e-15]
 [-3.11356605e-15  2.29834516e-15  3.34700373e+01]]
[3.91737156e+00 3.91737156e+00 4.79739654e-16]
rotation matrix = [[ 0.60493827  0.39506173  0.69135802]
 [ 0.39506173  0.60493827 -0.69135802]
 [-0.69135802  0.69135802  0.20987654]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -4  4]
 [ 1  3 -4]
 [ 0  4  7]]; sigma_1 = 81

U2 = 
[[ 1 -4 -4]
 [ 1  3  4]
 [ 0 -4  7]]; sigma_2 = 81

D = 
[[ 1.  0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156e+00 3.91737156e+00 4.79739654e-16] ; theta = 77.88

[[ 1.16932065e+02  1.76064541e-15 -9.17818103e-16]
 [ 7.28446964e-16  2.21599999e+01 -5.12268991e-15]
 [-1.94600755e-15 -1.09605055e-16  2.75611519e+01]]
[ 1.95868578 -1.95868578 27.4216009 ]
[-1.95868578e+00 -1.95868578e+00 -4.37456136e-16]
[-27.4216009   27.4216009    3.91737156]
[[  7   1  -1]
 [ -8   0 -13]
 [ -6   0  15]]
[[  7   1   1]
 [ -6   0 -15]
 [ -8   0  13]]
num of atoms1 = 198
[[ 1.10244608e+02 -4.06971354e-15  1.38881786e-15]
 [-1.32545884e-16  2.21599999e+01 -9.69894718e-16]
 [-8.95341490e-16  1.57114276e-17  3.89773549e+01]]
Warning!, this programme will rewrite the POSCAR file in this dir!
Warning!, this programme will rewrite the POSCAR file in this dir!
[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.66666667 -0.33333333  0.66666667]
 [ 0.66666667  0.66666667 -0.33333333]
 [-0.33333333  0.66666667  0.66666667]]
Congrates, we found an appx CSL!

U1 = 
[[-1  1  1]
 [ 0 -1  1]
 [ 1  0  1]]; sigma_1 = 3

U2 = 
[[-1  0  1]
 [ 1 -1  1]
 [ 0  1  1]]; sigma_2 = 3

[[ 1.06851650e+02  1.72991013e-16 -5.71141511e-15]
 [ 1.00158358e-14  2.03552597e+01  4.78843338e-15]
 [-1.60621810e-15 -2.32355542e-15  3.08454145e+01]]
[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.75675676 -0.32432432  0.56756757]
 [ 0.56756757  0.75675676 -0.32432432]
 [-0.32432432  0.56756757  0.75675676]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  0  3]
 [ 1 -3 -4]
 [ 1  4  0]]; sigma_1 = 37

U2 = 
[[ 1 -3  0]
 [ 1  0 -4]
 [ 1  4  3]]; sigma_2 = 37

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.91737156] ; theta = 50.56999199

[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.75675676 -0.32432432  0.56756757]
 [ 0.56756757  0.75675676 -0.32432432]
 [-0.32432432  0.56756757  0.75675676]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  4  3]
 [ 1 -3  0]
 [ 1  0 -4]]; sigma_1 = 37

U2 = 
[[ 1  4  0]
 [ 1  0  3]
 [ 1 -3 -4]]; sigma_2 = 37

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.917

[[ 1.17814583e+02  3.77199887e-15 -1.72208789e-15]
 [ 1.07740505e-14  2.03552597e+01  2.89551283e-15]
 [-8.10194484e-16 -4.75787058e-16  2.26734271e+01]]
[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.98630137 -0.10958904  0.12328767]
 [ 0.12328767  0.98630137 -0.10958904]
 [-0.10958904  0.12328767  0.98630137]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -6 -2]
 [ 1  2 -3]
 [ 1  3  6]]; sigma_1 = 73

U2 = 
[[ 1 -6 -3]
 [ 1  3 -2]
 [ 1  2  6]]; sigma_2 = 73

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.91737156] ; theta = 11.63505119

[3.91737156 3.91737156 3.91737156]
rotation matrix = [[ 0.98630137 -0.10958904  0.12328767]
 [ 0.12328767  0.98630137 -0.10958904]
 [-0.10958904  0.12328767  0.98630137]]
Congrates, we found an appx CSL!

U1 = 
[[ 1  6 -2]
 [ 1 -3  6]
 [ 1 -2 -3]]; sigma_1 = 73

U2 = 
[[ 1  6 -3]
 [ 1 -2  6]
 [ 1 -3 -2]]; sigma_2 = 73

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.91737156 3.91737156 3.917

In [ ]:
max_sigma = 100

    element = Element(name)
    axis = list(map(int, list(axis_name)))
    get_all_STGBs(axis,element,max_sigma)

In [93]:
hkl_1, hkl_2

(array([0, 2, 1]), array([ 0,  5, -1]))

In [94]:
np.dot(my_interface2.lattice_1.T,hkl_1.T).T

array([ 4.27650024, -4.27650024,  1.42550008])

In [96]:
MID(my_interface2.lattice_1,n_1)

array([0, 2, 1])

In [97]:
dot(my_interface2.lattice_1,array([0, 2, 1]))

array([ 1.42550008, -1.42550008, -4.27650024])

In [89]:
my_interface2.lattice_1

array([[ 1.42550008,  1.42550008, -1.42550008],
       [ 1.42550008, -1.42550008,  1.42550008],
       [ 1.42550008, -1.42550008, -1.42550008]])

In [99]:
a = np.array([[ -1.42550008,  1.42550008, -1.42550008],
       [- 1.42550008, -1.42550008,  1.42550008],
       [ 1.42550008, -1.42550008, -1.42550008]])

In [100]:
MID(a,n_1)

array([ 3, -2, -1])

In [101]:
dot(a,array([ 3, -2, -1]))

array([-5.70200032, -2.85100016,  8.55300048])

In [13]:
my_interface2.compute_stgb(axis, n_2, lim = 10, tol = 1e-5)
my_interface2.get_bicrystal(xyz_1 = [1,1,1], xyz_2 =[1,1,1],filename = 'atominfile', filetype='LAMMPS',mirror = False)

[ 9.04999999 -1.81       -7.23999999]
[-3.62 -3.62 -3.62]
[ 1.81       -5.42999999  3.62      ]
[[-4  1 -2]
 [-1  1  3]
 [ 5  1 -1]]
[[-1  1 -3]
 [-4  1  2]
 [ 5  1  1]]
num of atoms1 = 42
[[ 1.17301406e+01 -6.10377046e-16 -4.40385647e-16]
 [ 3.56509316e-16  6.27002391e+00 -5.22229576e-16]
 [ 1.99435554e-16  3.21762382e-16  6.77239986e+00]]


In [62]:
my_interface2.CSL

array([[ 1.42550008, -4.27650024,  4.27650024],
       [ 1.42550008,  1.42550008, -4.27650024],
       [ 1.42550008,  4.27650024,  1.42550008]])

In [63]:
my_interface.CSL

array([[ 2.85100016e+00, -5.70200032e+00,  1.74573411e-16],
       [ 2.85100016e+00,  0.00000000e+00, -2.85100016e+00],
       [ 2.85100016e+00,  2.85100016e+00,  5.70200032e+00]])

In [27]:
my_interface2.CNID

array([[ 3.62      , -9.04999999],
       [ 3.62      ,  5.42999999],
       [ 3.62      ,  3.62      ]])

In [28]:
15.779214153847805*5.119453088727666

80.78094663761145

In [29]:
6.27002391e+00*1.11575893e+01

69.95835168896016

In [30]:
-0.19973688802338993*79

-15.779214153847805

In [31]:
-0.196902041874141*13*2

-5.119453088727666

In [120]:
CNID = dot(my_interface2.orient, my_interface2.CNID)

In [121]:
print(CNID)

[[2.80565431e-16 2.10005904e-16]
 [2.46903857e+00 2.53688405e-15]
 [3.86913167e-16 1.06674658e+01]]


In [34]:
0.1959382473359075*32

6.27002391474904

In [38]:
34*np.array([0.1959382473359075,-0.061470822693617914])

array([ 6.66190041, -2.09000797])

In [118]:
norm(my_interface2.conv_lattice_1[:,0])/2*np.sqrt(3)

2.469038565448138

In [6]:
a = np.array([1,1,1])
b = np.array([2,2,2])
c = np.array([3,3,3])
np.column_stack((a,b,c))

array([[1, 2, 3],
       [1, 2, 3],
       [1, 2, 3]])

In [5]:
my_interface3 = core2('Cu_mp-30_conventional_standard.cif','Cu_mp-30_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


In [6]:
def get_a_b(CSL, axis):
    hkl_perp_axis = MID(CSL, axis)
    a, b = get_pri_vec_inplane(hkl_perp_axis, CSL).T

    if(norm(cross(axis,[1,0,0])) < 1e-8):
        b = a + b
    elif (norm(cross(axis,[1,1,1])) < 1e-8):
        if dot(a,b) < 0:
            b = a + b
        b = a + b
    if (abs(norm(a) - norm(b)) < 1e-8):
        raise RuntimeError ('the tow vectors are identical!')

    return a.T, b.T

In [7]:
def get_STGB_MLs(CSL, n_1, n_2):
    hkl_1 = MID(CSL, n_1)
    hkl_2 = MID(CSL, n_2)

    return hkl_1, hkl_2

In [ ]:
# axis = np.array([1,1,1])
# hkl_perp_axis = MID(my_interface.CSL, np.array([1,1,1]))
hkl_perp_axis = MID(my_interface.CSL, axis)
a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T
# b = a+b
hkl_1 = MID(my_interface.lattice_1,a)
hkl_2 = MID(my_interface.lattice_1,b)

In [55]:
def get_expansion_xyz(cell):
    exp_x = ceil(100/norm(cell[:,0]))
    exp_y = ceil(20/norm(cell[:,1]))
    exp_z = ceil(20/norm(cell[:,2]))
    return exp_x, exp_y, exp_z

In [52]:
def define_bicrystal_regions(xhi):
    tol = 1e-5
    """
    generate a file defining some regions in the LAMMPS and define the atoms
    inside these regions into some groups.
    argument:
    region_names --- list of name of regions
    region_los --- list of the low bounds
    region_his --- list of the hi bounds
    """
    end_fixbulk1 = xhi/2-30
    start_fixbulk2 = xhi/2+30
    start_middle = xhi/2-20
    end_middle = xhi/2+20
    start_right = xhi/2 + tol
    start_bulk = 160
    end_bulk = 165


    with open('blockfile', 'w') as fb:
        fb.write('region fixbulk1 block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(end_fixbulk1))
        fb.write('region fixbulk2 block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
        format(start_fixbulk2))
        fb.write('region middle block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_middle,end_middle))
        fb.write('region right block {0:.16f} EDGE EDGE EDGE EDGE EDGE units box \n'.\
        format(start_right))
        fb.write('region left block EDGE {0:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_right))
        fb.write('region bulk block {0:.16f} {1:.16f} EDGE EDGE EDGE EDGE units box \n'.\
        format(start_bulk,end_bulk))
        fb.write('group fixbulk1 region fixbulk1 \n')
        fb.write('group fixbulk2 region fixbulk2 \n')
        fb.write('group middle region middle \n')
        fb.write('group right region right \n')
        fb.write("group left region left \n")
        fb.write('group bulk region bulk \n')

In [53]:
def define_bicrystal_regions2(xhi):
    half_cell = xhi/2
    text = f"""

displace_atoms right move ${{tx}} ${{ty}} ${{tz}} units box

group left_dash region left 

group deleted_atoms subtract left_dash left

displace_atoms deleted_atoms move {half_cell} 0 0 units box

group fixbulk1_dash region fixbulk1 
group fixbulk2_dash region fixbulk2 
group middle_dash region middle 
group bulk_dash region bulk
            """
    with open("blockfile2", "w") as f:
        f.writelines(text)

In [8]:
axis = np.array([1,1,1])
max_sigma = 100
eps = 1e-5

element = Element("Cu")
cif_filename = "Cu_mp-30_conventional_standard.cif"
my_interface = core(cif_filename, cif_filename)
my_interface.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
factor = element.getLatP()/(norm(my_interface.conv_lattice_1[:,0]))
my_interface.lattice_1 =  my_interface.lattice_1*factor
my_interface.lattice_2 =  my_interface.lattice_2*factor
my_interface.conv_lattice_1 =  my_interface.conv_lattice_1*factor
my_interface.conv_lattice_2 =  my_interface.conv_lattice_2*factor  
my_interface.lattice_1 = my_interface.conv_lattice_1
my_interface.lattice_2 = my_interface.conv_lattice_2




Warning!, this programme will rewrite the POSCAR file in this dir!


In [9]:
axis

array([1, 1, 1])

In [9]:
cif_filename = "Cu_mp-30_conventional_standard.cif"
my_interface2 = core(cif_filename, cif_filename)
my_interface2.parse_limit(du = 1e-4, S  =  1e-4, sgm1=max_sigma, sgm2=max_sigma, dd = 1e-4)
factor = element.getLatP()/(norm(my_interface2.conv_lattice_1[:,0]))
my_interface2.lattice_1 =  my_interface2.lattice_1*factor
my_interface2.lattice_2 =  my_interface2.lattice_2*factor
my_interface2.conv_lattice_1 =  my_interface2.conv_lattice_1*factor
my_interface2.conv_lattice_2 =  my_interface2.conv_lattice_2*factor
axis_cart = dot(my_interface2.conv_lattice_1, axis)

axis_pri = np.array(np.round(dot(inv(my_interface2.lattice_1), axis_cart),8),dtype = int)

Warning!, this programme will rewrite the POSCAR file in this dir!


In [10]:
axis = np.array([1,1,1])

# ここではconventional cellをmy_interface.lattice_1にして計算をしている
# axis = np.array([1,1,1])
a,b = get_pri_vec_inplane(axis, np.array(np.round(dot(inv(my_interface.lattice_1), my_interface.lattice_1),8),dtype = int)).T
cell = np.column_stack((axis,a,b))
my_interface.atoms_1, my_interface.elements_1,my_interface.lattice_1 = super_cell(cell,my_interface.lattice_1,my_interface.atoms_1,my_interface.elements_1)
my_interface.atoms_2, my_interface.elements_2,my_interface.lattice_2 = my_interface.atoms_1.copy(), my_interface.elements_1.copy(),my_interface.lattice_1.copy()
my_interface.lattice_2_TD = my_interface.lattice_1.copy()

In [11]:
# axis = np.array([1,1,1])
axis_cart = dot(my_interface2.conv_lattice_1, axis)

axis_pri = np.array(np.round(dot(inv(my_interface2.lattice_1), axis_cart),8),dtype = int)
# ここではconventional cellをmy_interface.lattice_1にして計算をしている
# axis = np.array([1,1,1])
a,b = get_pri_vec_inplane(axis_pri, np.array(np.round(dot(inv(my_interface2.lattice_1), my_interface2.lattice_1),8),dtype = int)).T
cell = np.column_stack((axis_pri,a,b))
my_interface2.atoms_1, my_interface2.elements_1,my_interface2.lattice_1 = super_cell(cell,my_interface2.lattice_1,my_interface2.atoms_1,my_interface2.elements_1)
my_interface2.atoms_2, my_interface2.elements_2,my_interface2.lattice_2 = my_interface2.atoms_1.copy(), my_interface2.elements_1.copy(),my_interface2.lattice_1.copy()
my_interface2.lattice_2_TD = my_interface2.lattice_1.copy()

In [54]:
def get_thetas(axis,max_sigma):
    theta_list = []
    sigma_list = []
    lists, thetas = getsigmas(axis, max_sigma)
    return thetas

In [43]:
getsigmas([1,1,1], max_sigma)

([3, 7, 13, 19, 31, 37, 43, 49, 61, 67, 73, 79, 91, 97],
 [59.99999999999999,
  38.21321070173819,
  27.79577249602797,
  46.82644889274108,
  17.896551129254163,
  50.569992092103575,
  15.17817893794989,
  43.573578596523625,
  52.659006983369785,
  24.432697679454535,
  11.635051288887157,
  33.99217611435437,
  10.41743820571022,
  30.590688602809458])

In [120]:
os.getcwd()

'C:\\Users\\hatay\\calc\\220418\\test_program'

In [119]:
os.chdir("..")

In [18]:
theta = 38.21321070173819
my_interface2.search_one_position(axis_pri,theta-0.001,1,0.001)

[3.62 3.62 3.62]
rotation matrix = [[ 0.85715005 -0.28570997  0.42855991]
 [ 0.42855991  0.85715005 -0.28570997]
 [-0.28570997  0.42855991  0.85715005]]
Congrates, we found an appx CSL!

U1 = 
[[ 2  0  1]
 [-1 -2  0]
 [ 0  1 -2]]; sigma_1 = 7

U2 = 
[[ 2  1  0]
 [ 0 -2  1]
 [-1  0 -2]]; sigma_2 = 7

D = 
[[ 1.000e+00 -1.008e-05  1.008e-05]
 [ 1.008e-05  1.000e+00 -1.008e-05]
 [-1.008e-05  1.008e-05  1.000e+00]]

axis = [3.62 3.62 3.62] ; theta = 38.2122107



In [19]:
my_interface2.CSL

array([[-1.81,  3.62, -1.81],
       [-3.62, -1.81,  1.81],
       [ 1.81,  1.81,  3.62]])

In [31]:
# 自分が作ったLatticeでの軸、いまはｘ軸が(1,1,1)のLatticeを作って、その軸の周りに回転させようとしているから[1,0,0]
# つまりsuper cellでの回転軸は常に(1,0,0)
rotation_axis = [1,0,0]

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
# my_interface.search_one_position(rotation_axis,38.21,1,0.01)

axis_list = axis.tolist()

# thetas = get_thetas(axis_list,max_sigma)


# for theta in thetas:
theta = 38.21321070173819
my_interface.search_one_position(rotation_axis,theta-0.001,1,0.001)

#     my_interface.search_one_position(axis,theta_list[i][j]-0.001,1,0.001)
# CSL = my_interface.CSL
hkl_perp_axis = MID(my_interface.CSL, axis)
# a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T

a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T
print(a)
print(b)

if(norm(cross(axis,[1,0,0])) < 1e-8):
    b = a + b
elif (norm(cross(axis,[1,1,1])) < 1e-8):
    if dot(a,b) < 0:
        b = a + b
    b = a + b
if (abs(norm(a) - norm(b)) < 1e-8):
    raise RuntimeError ('the tow vectors are identical!')
    
print(a)
print(b)

hkl_1 = MID(my_interface2.lattice_1,a.T)
hkl_2 = MID(my_interface2.lattice_1,b.T)

print(hkl_1)
print(hkl_2)

# hkl_1, hkl_2 = get_STGB_MLs(my_interface.lattice_1, n_1, n_2)

# hkl_name_1 = get_primitive_hkl(hkl_1, my_interface.lattice_1, my_interface.conv_lattice_1)
# hkl_name_2 = get_primitive_hkl(hkl_2, my_interface.lattice_2, my_interface.conv_lattice_2)

# get_gb_files(my_interface,axis.tolist(),hkl_1,element)


# get_gb_files(my_interface,axis.tolist(),hkl_2,element)

[3.62 3.62 3.62]
rotation matrix = [[ 0.85715005 -0.28570997  0.42855991]
 [ 0.42855991  0.85715005 -0.28570997]
 [-0.28570997  0.42855991  0.85715005]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  0]
 [ 0  3 -2]
 [ 0 -1  3]]; sigma_1 = 7

U2 = 
[[-1  0  0]
 [ 0  2 -3]
 [ 0  1  2]]; sigma_2 = 7

D = 
[[ 1.000e+00 -1.008e-05  1.008e-05]
 [ 1.008e-05  1.000e+00 -1.008e-05]
 [-1.008e-05  1.008e-05  1.000e+00]]

axis = [3.62 3.62 3.62] ; theta = 38.2122107

[  3.62       -10.85999999   7.23999999]
[-7.23999999 -3.62       10.85999999]
[  3.62       -10.85999999   7.23999999]
[ -3.62       -14.47999998  18.09999998]
[ 2 -3  1]
[ 5 -4 -1]


In [30]:
hkl_1

array([ 2, -3,  1])

In [28]:
get_gb_files(my_interface2,[1,1,1],hkl_1,element)

cell 1:
[[ 2 -1 -5]
 [-3  0 -3]
 [ 1  2  1]]
cell 2:
[[ 2 -1 -5]
 [-3  0 -3]
 [ 1  2  1]]
x, y, z = 15.0, 5.0, 2.0
[[ 2 -1 -5]
 [-3  0 -3]
 [ 1  2  1]]
[[ 3  0 -3]
 [-2 -1 -5]
 [-1  2  1]]
num of atoms1 = 42
[[ 1.01585998e+02  1.92857071e-15  1.05695185e-15]
 [-2.97244434e-15  2.21678821e+01  4.61495396e-15]
 [-2.59707533e-15  1.08899667e-15  3.31778480e+01]]


In [33]:
my_interface2.compute_bicrystal(hkl_1, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-5)
my_interface2.get_bicrystal(xyz_1 = [1,1,1], xyz_2 =[1,1,1],filename = 'atominfile', filetype='LAMMPS',mirror = False)

cell 1:
[[ 2 -1 -5]
 [-3  0 -3]
 [ 1  2  1]]
cell 2:
[[ 2 -1 -5]
 [-3  0 -3]
 [ 1  2  1]]
[[ 2 -1 -5]
 [-3  0 -3]
 [ 1  2  1]]
[[ 3  0 -3]
 [-2 -1 -5]
 [-1  2  1]]
num of atoms1 = 42
[[ 6.77239986e+00 -9.97177769e-17 -1.69197013e-15]
 [ 4.48424568e-17  4.43357643e+00 -3.99246753e-16]
 [ 2.21022325e-16  1.51363716e-17  1.65889240e+01]]


In [11]:
theta = 38.21321070173819

In [16]:
n_1

array([  7.23999999,   3.62      , -10.85999999])

In [12]:
my_interface2.CSL

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [12]:
my_interface.search_one_position(axis,theta-0.0000001,1,0.0000001)
my_interface2.search_one_position(axis_pri,theta-0.0000001,1,0.0000001)

CSL = my_interface.CSL
n_1, n_2 = get_a_b(my_interface.CSL, dot(my_interface.lattice_1,axis))
hkl_1, hkl_2 = get_STGB_MLs(my_interface2.lattice_1, n_1, n_2)

[3.62 3.62 3.62]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 1 -2  0]
 [ 1  0 -1]
 [ 1  1  2]]; sigma_1 = 7

U2 = 
[[ 1 -2 -1]
 [ 1  1  0]
 [ 1  0  2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 38.2132106

[3.62 3.62 3.62]
rotation matrix = [[ 0.85714286 -0.28571429  0.42857143]
 [ 0.42857143  0.85714286 -0.28571429]
 [-0.28571429  0.42857143  0.85714286]]
Congrates, we found an appx CSL!

U1 = 
[[ 2  0  1]
 [-1 -2  0]
 [ 0  1 -2]]; sigma_1 = 7

U2 = 
[[ 2  1  0]
 [ 0 -2  1]
 [-1  0 -2]]; sigma_2 = 7

D = 
[[ 1. -0.  0.]
 [ 0.  1. -0.]
 [-0.  0.  1.]]

axis = [3.62 3.62 3.62] ; theta = 38.2132106



In [17]:
print(hkl_1)
3.62 1.81 -5.43

[0 1 5]


In [14]:
my_interface2.compute_bicrystal(hkl_1, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-5)
my_interface2.get_bicrystal(xyz_1 = [1,1,1], xyz_2 =[1,1,1],filename = 'atominfile', filetype='LAMMPS',mirror = False)

[ 3.62        1.81       -5.42999999]
cell 1:
[[-3  0 -3]
 [ 1  2  1]
 [ 2 -1 -5]]
cell 2:
[[-3  0 -3]
 [ 1  2  1]
 [ 2 -1 -5]]
[[-3  0 -3]
 [ 1  2  1]
 [ 2 -1 -5]]
[[-2 -1 -5]
 [-1  2  1]
 [ 3  0 -3]]
num of atoms1 = 42
[[ 6.77239986e+00  2.96832938e-16 -1.18733175e-15]
 [ 2.58689121e-16  4.43357643e+00  1.45860197e-15]
 [-7.97272796e-17  1.51363716e-17  1.65889240e+01]]


In [27]:
from interface_generator import cross_plane

In [98]:
def cross_plane2(lattice, n, lim, orthogonal, tol):
    """
    get a primitive lattice vector cross a plane
    argument:
    lattice --- lattice matrix
    n --- a normal vector
    lim --- control how many vectors to be generated
    tol --- tolerance judging orthogonal
    return:
    a primitve vector normal to the plane
    """
    x = np.arange(-lim, lim, 1)
    y = x
    z = x
    indice = (np.stack(np.meshgrid(x, y, z)).T).reshape(len(x) ** 3, 3)
    indice_0 = indice[np.where(np.sum(abs(indice), axis=1) != 0)[0]]
    indice_0 = indice_0[np.where(np.gcd.reduce(indice_0, axis=1) == 1)[0]]
    ltc_p = dot(indice_0, lattice.T)
    ltc_p = ltc_p[np.argsort(norm(ltc_p, axis=1))]
    dot_list = get_ang_list(ltc_p, n)
    if orthogonal == False:
        normal_v = ltc_p[np.where(dot_list >= 0.5)[0]]
        normal_v = normal_v[np.argsort(norm(normal_v, axis=1))]
        normal_v = normal_v[0]
    else:
        try:
            normal_v = ltc_p[np.where(abs(dot_list - 1) < tol)[0]]
#             normal_v = ltc_p[np.where(abs(dot_list - 1) < tol)[0]]
#             normal_v = normal_v[np.argsort(norm(normal_v, axis=1))]
#             normal_v = normal_v[0]
        except:
            raise RuntimeError('failed to find a vector cross the plane. try larger lim or smaller tol or use non-orthogonal cell')
        normal_v = normal_v[np.argsort(norm(normal_v, axis=1))]
        normal_v = normal_v[0]
    return normal_v

In [99]:
def get_ang_list(m1, n):
    """
    compute a list of ang cos between one list of vecor and one vector
    """
    return 1 / norm(n) * abs(dot(m1, n)) / norm(m1, axis = 1)

In [100]:
v3 = cross_plane2(lattice=my_interface2.CSL, n=n_2, lim=50, orthogonal=True, tol=1e-5)

In [101]:
print(my_interface2.CSL) # CSLの基底3つ
print(n_2) # 入力したベクトル
print(v3) # cross plane で発見したベクトル
print(norm(n_2))

[[-1.81  3.62 -1.81]
 [-3.62 -1.81  1.81]
 [ 1.81  1.81  3.62]]
[ 18.09999998  -3.62       -14.47999998]
[-9.04999999  1.81        7.23999999]
23.460281295870015


In [35]:
v3 = cross_plane(lattice=my_interface2.CSL, n=n_2, lim=2, orthogonal=True, tol=1e-5)

In [130]:
# 自分が作ったLatticeでの軸、いまはｘ軸が(1,1,1)のLatticeを作って、その軸の周りに回転させようとしているから[1,0,0]
# つまりsuper cellでの回転軸は常に(1,0,0)
rotation_axis = [1,0,0]

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
# my_interface.search_one_position(rotation_axis,38.21,1,0.01)

axis_list = axis.tolist()

thetas = get_thetas(axis_list,max_sigma)


for theta in thetas:
    my_interface.search_one_position(rotation_axis,theta-0.001,1,0.001)

    #     my_interface.search_one_position(axis,theta_list[i][j]-0.001,1,0.001)
    # CSL = my_interface.CSL
    hkl_perp_axis = MID(my_interface.CSL, axis)
    # a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T

    a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T

    if(norm(cross(axis,[1,0,0])) < 1e-8):
        b = a + b
    elif (norm(cross(axis,[1,1,1])) < 1e-8):
        if dot(a,b) < 0:
            b = a + b
        b = a + b
    if (abs(norm(a) - norm(b)) < 1e-8):
        raise RuntimeError ('the tow vectors are identical!')

    hkl_1 = MID(my_interface.lattice_1,a.T)
    hkl_2 = MID(my_interface.lattice_1,b.T)

    # hkl_1, hkl_2 = get_STGB_MLs(my_interface.lattice_1, n_1, n_2)

    # hkl_name_1 = get_primitive_hkl(hkl_1, my_interface.lattice_1, my_interface.conv_lattice_1)
    # hkl_name_2 = get_primitive_hkl(hkl_2, my_interface.lattice_2, my_interface.conv_lattice_2)

    get_gb_files(my_interface,axis.tolist(),hkl_1,element)


    get_gb_files(my_interface,axis.tolist(),hkl_2,element)

[3.62 3.62 3.62]
rotation matrix = [[ 0.66667674 -0.33333333  0.66665659]
 [ 0.66665659  0.66667674 -0.33333333]
 [-0.33333333  0.66665659  0.66667674]]
Congrates, we found an appx CSL!

U1 = 
[[ 0  0  1]
 [ 1  1  0]
 [ 0 -1  0]]; sigma_1 = 1

U2 = 
[[0 0 1]
 [0 1 0]
 [1 0 0]]; sigma_2 = 1

D = 
[[ 1.000e+00 -1.008e-05  1.008e-05]
 [ 1.008e-05  1.000e+00 -1.008e-05]
 [-1.008e-05  1.008e-05  1.000e+00]]

axis = [3.62 3.62 3.62] ; theta = 59.999

cell 1:
[[ 0  1  0]
 [-1  0  1]
 [ 0  0 -2]]
cell 2:
[[ 0  1  0]
 [-1  0  1]
 [ 0  0 -2]]
x, y, z = 20.0, 4.0, 3.0
[[ 0  1  0]
 [-1  0  1]
 [ 0  0 -2]]
[[ 0  1  0]
 [ 0  0  2]
 [-1  0 -1]]
num of atoms1 = 24
[[ 1.02389062e+02 -1.08916015e-15  4.44577745e-15]
 [ 6.49579434e-16  2.50800957e+01  8.62940323e-16]
 [ 2.54827121e-15  1.21945394e-15  2.66014586e+01]]
cell 1:
[[ 0  0  1]
 [ 1  1  0]
 [ 1 -1  0]]
cell 2:
[[ 0  0  1]
 [ 1  1  0]
 [ 1 -1  0]]
x, y, z = 12.0, 4.0, 4.0
[[ 0  0  1]
 [ 1  1  0]
 [ 1 -1  0]]
[[ 0  0  1]
 [-1  1  0]
 [ 2  0  0]]


[[ 1.16291619e+02  6.73679813e-15 -1.33975695e-15]
 [-6.27646880e-16  2.50800957e+01 -7.25599873e-16]
 [-8.68579796e-16  2.11927424e-15  3.35704989e+01]]
[3.62 3.62 3.62]
rotation matrix = [[ 0.81633455 -0.30611916  0.48978461]
 [ 0.48978461  0.81633455 -0.30611916]
 [-0.30611916  0.48978461  0.81633455]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  0]
 [ 0 -8 -3]
 [ 0  3 -5]]; sigma_1 = 49

U2 = 
[[-1  0  0]
 [ 0 -5  3]
 [ 0 -3 -8]]; sigma_2 = 49

D = 
[[ 1.000e+00 -1.008e-05  1.008e-05]
 [ 1.008e-05  1.000e+00 -1.008e-05]
 [-1.008e-05  1.008e-05  1.000e+00]]

axis = [3.62 3.62 3.62] ; theta = 43.5725786

cell 1:
[[ 0 -1  0]
 [-8  0 -2]
 [ 3  0 13]]
cell 2:
[[ 0 -1  0]
 [-8  0 -2]
 [ 3  0 13]]
x, y, z = 3.0, 4.0, 1.0
[[ 0 -1  0]
 [-8  0 -2]
 [ 3  0 13]]
[[  0  -1   0]
 [ -5   0 -11]
 [ -3   0  13]]
num of atoms1 = 1176
[[ 1.07508515e+02  3.53386229e-15 -7.47062342e-15]
 [-1.18662507e-15  2.50800957e+01  9.80113064e-16]
 [-3.72575761e-15 -1.43151347e-15  6.20700700e+01]]
cell 1:
[[

[[ 1.00841531e+02 -4.75720717e-15 -2.17516741e-15]
 [ 5.60338404e-15  2.50800957e+01 -2.47641247e-15]
 [-8.19558093e-15  1.30339143e-15  8.73313276e+01]]
cell 1:
[[  0  -1   0]
 [-14   0  -8]
 [ -5   0  11]]
cell 2:
[[  0  -1   0]
 [-14   0  -8]
 [ -5   0  11]]
x, y, z = 2.0, 4.0, 1.0
[[  0  -1   0]
 [-14   0  -8]
 [ -5   0  11]]
[[  0  -1   0]
 [ -5   0 -11]
 [-14   0   8]]
num of atoms1 = 2328
[[ 1.74662655e+02 -3.50562262e-15  2.25187496e-15]
 [ 5.34268614e-15  2.50800957e+01 -8.33636766e-15]
 [ 7.99214289e-15 -9.37962745e-16  5.04207655e+01]]


In [27]:
def get_gb_files(interface,axis,hkl,element):
    interface.compute_bicrystal(hkl, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-5)
    half_cell = dot(interface.lattice_1, interface.bicrystal_U1)
    x,y,z = get_expansion_xyz(half_cell)
    print(f"x, y, z = {x}, {y}, {z}")
    xhi = 2*x*norm(half_cell[:,0])
    
    hkl_name = get_primitive_hkl(hkl, my_interface.lattice_1, my_interface.conv_lattice_1)
    hkl_name = np.array(hkl_name, dtype=int)
    hkl_name_num = hkl_name.copy()
    hkl_name = np.array(np.sort(np.abs(hkl_name))[::-1]/np.gcd.reduce(np.sort(np.abs(hkl_name))[::-1]),dtype=int)
    hkl_x, hkl_y, hkl_z = hkl_name
    dirname = f"{str(hkl_x)}_{str(hkl_y)}_{str(hkl_z)}"
    os.makedirs(dirname,exist_ok=True)
    os.chdir(dirname) 
    
    if (axis == [1,1,1]):    
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False)
    else:
        interface.get_bicrystal(xyz_1 = [x,y,z], xyz_2 =[x,y,z],filename = 'atominfile', filetype='LAMMPS',mirror = False)

    define_bicrystal_regions(xhi)
    define_bicrystal_regions2(xhi)
    
    with open(f"{element.element}.meam","w") as f:
        f.writelines(element.meam())

    with open("library.meam","w") as f:
        f.writelines(element.library_meam())

    with open("proto.in","w") as f:
        f.writelines(element.proto())


    os.chdir(os.pardir)

In [7]:
axis = np.array([1,1,1])

In [8]:
print_list(axis,100)

Sigma:     1  Theta:   0.00 
Sigma:     3  Theta:  60.00 
Sigma:     7  Theta:  38.21 
Sigma:    13  Theta:  27.80 
Sigma:    19  Theta:  46.83 
Sigma:    21  Theta:  21.79 
Sigma:    31  Theta:  17.90 
Sigma:    37  Theta:  50.57 
Sigma:    39  Theta:  32.20 
Sigma:    43  Theta:  15.18 
Sigma:    49  Theta:  43.57 
Sigma:    57  Theta:  13.17 
Sigma:    61  Theta:  52.66 
Sigma:    67  Theta:  24.43 
Sigma:    73  Theta:  11.64 
Sigma:    79  Theta:  33.99 
Sigma:    91  Theta:  10.42 
Sigma:    93  Theta:  42.10 
Sigma:    97  Theta:  30.59 


In [9]:
# ここではconventional cellをmy_interface.lattice_1にして計算をしている
# axis = np.array([1,1,1])
a,b = get_pri_vec_inplane(axis, np.array(np.round(dot(inv(my_interface.lattice_1), my_interface.lattice_1),8),dtype = int)).T
cell = np.column_stack((axis,a,b))
my_interface.atoms_1, my_interface.elements_1,my_interface.lattice_1 = super_cell(cell,my_interface.lattice_1,my_interface.atoms_1,my_interface.elements_1)
my_interface.atoms_2, my_interface.elements_2,my_interface.lattice_2 = my_interface.atoms_1.copy(), my_interface.elements_1.copy(),my_interface.lattice_1.copy()
my_interface.lattice_2_TD = my_interface.lattice_1.copy()

In [10]:
# 自分が作ったLatticeでの軸、いまはｘ軸が(1,1,1)のLatticeを作って、その軸の周りに回転させようとしているから[1,0,0]
# つまりsuper cellでの回転軸は常に(1,0,0)
rotation_axis = [1,0,0]
my_interface.parse_limit(du = 1e-4, S  = 1e-4, sgm1=100, sgm2=100, dd = 1e-4)

#Do searching!
#We already know that there is a CSL by rotation 180 degrees
my_interface.search_one_position(rotation_axis,38.21,1,0.01)

[3.621262 3.621262 3.621262]
rotation matrix = [[ 0.85716597 -0.28570042  0.42853445]
 [ 0.42853445  0.85716597 -0.28570042]
 [-0.28570042  0.42853445  0.85716597]]
Congrates, we found an appx CSL!

U1 = 
[[-1  0  0]
 [ 0  3  1]
 [ 0 -1  2]]; sigma_1 = 7

U2 = 
[[-1  0  0]
 [ 0  2 -1]
 [ 0  1  3]]; sigma_2 = 7

D = 
[[ 1.000e+00 -3.235e-05  3.235e-05]
 [ 3.235e-05  1.000e+00 -3.235e-05]
 [-3.235e-05  3.235e-05  1.000e+00]]

axis = [3.621262 3.621262 3.621262] ; theta = 38.21



In [11]:
# axis = np.array([1,1,1])
# hkl_perp_axis = MID(my_interface.CSL, np.array([1,1,1]))
hkl_perp_axis = MID(my_interface.CSL, axis)
a, b = get_pri_vec_inplane(hkl_perp_axis, my_interface.CSL).T
# b = a+b
hkl_1 = MID(my_interface.lattice_1,a)
hkl_2 = MID(my_interface.lattice_1,b)

In [12]:
my_interface.compute_bicrystal(hkl_1, normal_ortho = True, plane_ortho = True, lim = 50, tol = 1e-2)

cell 1:
[[ 0 -1  0]
 [ 3  0  1]
 [-1  0 -5]]
cell 2:
[[ 0 -1  0]
 [ 3  0  1]
 [-1  0 -5]]


In [13]:
my_interface.get_bicrystal(xyz_1 = [2,1,1], xyz_2 = [2,1,1],filetype="LAMMPS",filename="atominfile")

[[ 0 -1  0]
 [ 3  0  1]
 [-1  0 -5]]
[[ 0 -1  0]
 [ 2  0  4]
 [ 1  0 -5]]
num of atoms1 = 168
num of atoms2 = 168
[[ 2.70990434e+01  6.01710864e-16  6.91810869e-16]
 [-1.95023943e-15  6.27220977e+00 -4.40441118e-15]
 [-3.40457860e-16  5.29203675e-16  2.34684600e+01]]
